---
# install & load
---

[참고블로그](https://ratsgo.github.io/nlpbook/docs/ner/train/#%EC%BD%94%EB%93%9C7-%EB%A7%90%EB%AD%89%EC%B9%98-%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C)

In [1]:
!pip install transformers
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.6 MB/s eta 0:00:00


In [15]:
##### pytorch #####
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, RandomSampler


##### 시각화 #####
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns 

##### 기본 모듈 #####
import pandas as pd
import numpy as np
import os
import random
import json
import math
import easydict
from pprint import pprint
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

##### 디버깅 #####
import pdb

##### cuda #####
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # GPU 할당

##### 경고무시 #####
import warnings
warnings.filterwarnings(action='ignore')


import re

In [30]:
class Args:
  pretrained_model_name = 'beomi/kcbert-base'
  downstream_corpus_name =  'ner'
  downstream_model_dir = '/content/drive/MyDrive/2.Study/NER'
  downstream_corpus_root_dir = '/content/drive/MyDrive/2.Study/NER'
  downstream_task_name = 'named-entity-recognition'
  train_data = '/content/drive/MyDrive/2.Study/NER/train.txt'
  val_data = '/content/drive/MyDrive/2.Study/NER/val.txt'
  batch_size = 16
  learning_rate = 5e-5
  max_seq_length = 64
  epochs = 5
  seed = 7
  force_download = False
  overwrite_cache = False

args = Args()  

In [7]:
from ratsnlp.nlpbook.ner import NERTrainArguments
from ratsnlp import nlpbook

nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 17.9M/17.9M [00:00<00:00, 54.3MB/s]
Downloading: 100%|██████████| 1.13M/1.13M [00:00<00:00, 41.3MB/s]


---
# Tokenizer
---

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

- @dataclass 데코레이터를 사용해주는 이유?

  - 데코레이터를 정의하지 않으면 클래스의 모든 속성을 별도로 정의해야 하므로 코드의 길이가 더 길어짐
        class NERExample:
          def __init__(text, label)
            self.text = text
            self.label = label
  - 데코레이터를 사용하면 속성을 정의하고 생성자를 구현하는 작업이 간단해지므로 코드의 가독성이 높아짐

  - 또한 @dataclass를 사용하면 클래스의 인스턴스에 대한 여러 유용한 메서드들이 자동으로 생성됨              

---
# Dataset & DataLoader
---

In [9]:
from typing import List, Optional
from dataclasses import dataclass

@dataclass
class NERExample:
    text: str
    label: Optional[str] = None

@dataclass
class NERFeatures:
    input_ids: List[int]
    attention_mask: Optional[List[int]] = None
    token_type_ids: Optional[List[int]] = None
    label_ids: Optional[List[int]] = None

NER_CLS_TOKEN = "[CLS]"
NER_SEP_TOKEN = "[SEP]"
NER_PAD_TOKEN = "[PAD]"
NER_MASK_TOKEN = "[MASK]"
NER_PAD_ID = 2

class NERCorpus:

    def __init__(self, args: args):
        self.args = args

    # 파일에서 text와 label값을 추출하여 객체를 생성
    def get_examples(self, data_root_path, mode):
        data_fpath = os.path.join(data_root_path, f"{mode}.txt")
        examples = []
        for line in open(data_fpath, "r", encoding="utf-8").readlines():
            text, label = line.split("\u241E")
            examples.append(NERExample(text=text, label=label))
        return examples

    '''self.args 에 저장된 경로와 파일 이름 정보를 이용하여 
       label_map.txt 파일을 읽어오거나, 파일이 없으면 새로 생성 
       이때 train.txt 파일에서 regex_ner 패턴에 해당하는 문자열에서 추출한 
       ner_tag 값을 이용하여 labels 리스트를 생성하고, 이를 label_map.txt 파일에 저장
    '''
    def get_labels(self):
        label_map_path = os.path.join(
            self.args.downstream_model_dir, "label_map.txt",)
        
        if not os.path.exists(label_map_path):
            os.makedirs(self.args.downstream_model_dir, exist_ok=True)
            ner_tags = []
            regex_ner = re.compile('<(.+?):[A-Z]{3}>')
            train_corpus_path = os.path.join(
                self.args.downstream_corpus_root_dir,
                self.args.downstream_corpus_name,
                "train.txt",
            )
            target_sentences = [line.split("\u241E")[1].strip()
                                for line in open(train_corpus_path, "r", encoding="utf-8").readlines()]
            for target_sentence in target_sentences:
                regex_filter_res = regex_ner.finditer(target_sentence)
                for match_item in regex_filter_res:
                    ner_tag = match_item[0][-4:-1]
                    if ner_tag not in ner_tags:
                        ner_tags.append(ner_tag)
            b_tags = [f"B-{ner_tag}" for ner_tag in ner_tags]
            i_tags = [f"I-{ner_tag}" for ner_tag in ner_tags]
            labels = [NER_CLS_TOKEN, NER_SEP_TOKEN, NER_PAD_TOKEN, NER_MASK_TOKEN, "O"] + b_tags + i_tags
            with open(label_map_path, "w", encoding="utf-8") as f:
                for tag in labels:
                    f.writelines(tag + "\n")
        else:
            labels = [tag.strip() for tag in open(label_map_path, "r", encoding="utf-8").readlines()]
        return labels

    @property
    # 라벨의 개수
    def num_labels(self):
        return len(self.get_labels())

In [10]:
# 주어진 문장을 토큰화하고, NER 태그를 찾아서 해당 태그에 대한 정보를 추출
def _process_target_sentence(
        tokens: List[str],
        origin_sentence: str,
        target_sentence: str,
        max_length: int,
        label_map: dict,
        tokenizer: BertTokenizer,
        cls_token_at_end: Optional[bool] = False,
):
    """
    target_sentence = "―<효진:PER> 역의 <김환희:PER>(<14:NOH>)가 특히 인상적이었다."
    tokens = ["―", "효", "##진", "역", "##의", "김", "##환", "##희",
              "(", "14", ")", "가", "특히", "인상", "##적이", "##었다", "."]
    label_sequence = ['O', 'B-PER', 'I-PER', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O',
                      'B-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
    """

    ################# [UNK] 토큰을 처리하는 부분 #################
    if "[UNK]" in tokens:
        processed_tokens = []
        
        # 입력 문장을 기본 토크나이저를 이용해 토큰화
        basic_tokens = tokenizer.basic_tokenizer.tokenize(origin_sentence)
        # basic_tokens에서 하나씩 꺼내서 
        for basic_token in basic_tokens:
            # 현재 토큰(basic_token)을 서브 토크나이저를 이용해 다시 토큰화
            current_tokens = tokenizer.tokenize(basic_token)
            '''why? 왜 다시 서브토크나이저를 이용해 재토큰화를 하는가?
               
               [UNK] 토큰이 포핟묀 경우 해당 토큰을 서브 토큰으로 분리하기 위함
               이를 분리해야 모델이 이해할 수 있기 때문'''

            # 서브 토큰에도 [UNK] 토큰이 있다면 기본 토큰 그대로를 processed_tokens 리스트에 추가
            # 즉, 서브 토큰일 사용했음에도 [UNK] 토큰이 존재한다면 그냥 사용하겠다는 의미
            if "[UNK]" in current_tokens:
                processed_tokens.append(basic_token)
            # [UNK] 토큰이 없다면... 서브 토큰 리스트를 processed_tokens 리스트에 추가                 
            else:
                processed_tokens.extend(current_tokens)
    else:
        processed_tokens = tokens
    #############################################################

    # 토큰화된 결과(processed_tokens)의 각 토큰의 시작 인덱스와 전체 길이를 계산하기 위해 초기화
    prefix_sum_of_token_start_index, sum = [0], 0
    for i, token in enumerate(processed_tokens):
        # 토큰이 서브 토큰인 경우(##으로 시작) ##를 제외한 실제 토큰의 길이를 계산해서 sum에 추가
        if token.startswith("##"):
            sum += len(token) - 2
        else:
            sum += len(token)
        prefix_sum_of_token_start_index.append(sum)

    # 정규표현식 <ner_text:ner_tag> 형식으로 된 개체명 태그를 찾기 위한 패턴
    regex_ner = re.compile('<(.+?):[A-Z]{3}>')  # NER Tag가 2자리 문자면 {3} -> {2}로 변경 (e.g. LOC -> LC) 인경우
    # target_sentence에서 개체명 태그를 찾기
    regex_filter_res = regex_ner.finditer(target_sentence.replace(" ", ""))

    list_of_ner_tag = [] # 개체명 태그 
    list_of_ner_text = [] # 개체명 텍스트
    list_of_tuple_ner_start_end = [] # 개체명의 시작과 끝 인덱스 

    count_of_match = 0 # 정규식에 매칭되는 문자열의 개수를 세는 변수
    # 정규식에서 매칭되는 문자열을 하나씩 순회
    for match_item in regex_filter_res:
        ner_tag = match_item[0][-4:-1]  # <4일간:DUR> -> DUR
        ner_text = match_item[1]  # <4일간:DUR> -> 4일간
        
        # <,:,>,NER 태그명(3자리)를 제거한 값의 시작 인덱스와 끝 인덱스 계산
        start_index = match_item.start() - 6 * count_of_match 
        end_index = match_item.end() - 6 - 6 * count_of_match
        '''이전 매칭에서 <, :, NER 태그 이름, > 총 6개의 문자열이 사용되었으므로, 
          새로운 매칭에서는 해당 문자열의 개수를 곱해 이전 매칭에서 사용된 문자열을 제거할 수 있다. 
          이를 통해 각 NER 태그의 시작 위치와 끝 위치를 정확하게 계산할 수 있다.'''
        list_of_ner_tag.append(ner_tag)
        list_of_ner_text.append(ner_text)
        list_of_tuple_ner_start_end.append((start_index, end_index))
        count_of_match += 1

    ###################### 토큰의 시작 위치; 정보를 사용해 레이블 시퀀스를 생성하는 코드 ######################
    '''반복문에서 현재 토큰과 그 위치를 가져와 entity_index가 NER 태그 정보의 길이보다 작을 때까지 반복하며, 
       이전 엔티티의 범위보다 현재 토큰의 시작 위치가 큰 경우 다음 엔티티를 가져온다. 
       그리고 토큰의 시작 위치가 현재 엔티티의 범위 안에 포함되는 경우에는 엔티티의 태그를 생성하고, 
       현재 엔티티가 B 태그인지 아닌지 여부를 is_entity_still_B 변수로 확인하며, 레이블 시퀀스에 추가 
       포함되지 않는 경우에는 O 태그를 생성하고, is_entity_still_B 변수를 True로 초기화.'''
    label_sequence = [] # 레이블 시퀀스 초기화
    entity_index = 0 # 엔티티 인덱스 초기화
    is_entity_still_B = True # 엔티티가 첫번째 토큰에서 시작하는지 여부

    for tup in zip(processed_tokens, prefix_sum_of_token_start_index):
        token, index = tup

        if entity_index < len(list_of_tuple_ner_start_end):
            start, end = list_of_tuple_ner_start_end[entity_index]

            if end < index:  # 엔티티 범위보다 현재 seq pos가 더 크면 다음 엔티티를 꺼내서 체크
                is_entity_still_B = True
                entity_index = entity_index + 1 if entity_index + 1 < len(list_of_tuple_ner_start_end) else entity_index
                start, end = list_of_tuple_ner_start_end[entity_index]

            if start <= index and index < end:  # <13일:DAT>까지 -> ('▁13', 10, 'B-DAT') ('일까지', 12, 'I-DAT') 이런 경우가 포함됨, 포함 안시키려면 토큰의 length도 계산해서 제어해야함
                entity_tag = list_of_ner_tag[entity_index]
                if is_entity_still_B is True:
                    entity_tag = 'B-' + entity_tag
                    label_sequence.append(entity_tag)
                    is_entity_still_B = False
                else:
                    entity_tag = 'I-' + entity_tag
                    label_sequence.append(entity_tag)
            else:
                is_entity_still_B = True
                entity_tag = 'O'
                label_sequence.append(entity_tag)
        else:
            entity_tag = 'O'
            label_sequence.append(entity_tag)
    #############################################################################################

    # max_length 보다 크면 max_length -2 값까지만 자름
    label_sequence = label_sequence[:max_length - 2]

    # True라면 [CLS], [SEP] 토큰 끝에 추가 
    # False라면 [CLS], [SEP] 토큰 앞에 추가
    if cls_token_at_end:
        label_sequence = label_sequence + [NER_CLS_TOKEN, NER_SEP_TOKEN]
    else:
        label_sequence = [NER_CLS_TOKEN] + label_sequence + [NER_SEP_TOKEN]

    # padding
    pad_length = max(max_length - len(label_sequence), 0)
    pad_sequence = [NER_PAD_TOKEN] * pad_length
    label_sequence += pad_sequence

    # encoding
    label_ids = [label_map[label] for label in label_sequence]
    return label_ids

In [11]:
from transformers.tokenization_utils_base import PaddingStrategy, TruncationStrategy
from filelock import FileLock

# 모델 학습을 위해 입력 문장을 토큰화하고 레이블을 처리하여 input 값으로 사용할 수 있는 feature로 변환하는 함수
def _convert_examples_to_ner_features(
        examples: List[NERExample],
        tokenizer: BertTokenizer,
        args: args,
        label_list: List[str],
        cls_token_at_end: Optional[bool] = False,
    ):
        """
        `cls_token_at_end` define the location of the CLS token:
                - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
                - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        """
        # 레이블의 문자열을 정수로 매핑
        label_map = {label: i for i, label in enumerate(label_list)}
        # 레입ㄹ의 정수를 문자열로 매핑
        id_to_label = {i: label for i, label in enumerate(label_list)}

        # 입력 토큰, 마스크, 세그먼트, 레이블 인덱스 저장
        features = [] 
        for example in examples:
            tokens = tokenizer.tokenize(example.text)
            inputs = tokenizer._encode_plus(
                tokens,
                max_length=args.max_seq_length,
                truncation_strategy=TruncationStrategy.LONGEST_FIRST,
                padding_strategy=PaddingStrategy.MAX_LENGTH,
            )
            label_ids = _process_target_sentence(
                tokens=tokens,
                origin_sentence=example.text,
                target_sentence=example.label,
                max_length=args.max_seq_length,
                label_map=label_map,
                tokenizer=tokenizer,
                cls_token_at_end=cls_token_at_end,
            )
            features.append(NERFeatures(**inputs, label_ids=label_ids))

        return features

In [12]:
class NERDataset(Dataset):

    def __init__(
            self,
            args: args,
            tokenizer: BertTokenizer,
            corpus: NERCorpus,
            mode: Optional[str] = "train",
            convert_examples_to_features_fn=_convert_examples_to_ner_features,
    ):
        if corpus is not None:
            self.corpus = corpus
        else:
            raise KeyError("corpus is not valid")
        if not mode in ["train", "val", "test"]:
            raise KeyError(f"mode({mode}) is not a valid split name")
        # Load data features from cache or dataset file
        cached_features_file = os.path.join(
            args.downstream_corpus_root_dir,
            args.downstream_corpus_name,
            "cached_{}_{}_{}_{}_{}".format(
                mode,
                tokenizer.__class__.__name__,
                str(args.max_seq_length),
                args.downstream_corpus_name,
                args.downstream_task_name,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not args.overwrite_cache:
                self.features = torch.load(cached_features_file)

            else:
                corpus_path = os.path.join(
                    args.downstream_corpus_root_dir,
                    args.downstream_corpus_name,
                )
                examples = self.corpus.get_examples(corpus_path, mode)
                self.features = convert_examples_to_features_fn(
                    examples,
                    tokenizer,
                    args,
                    label_list=self.corpus.get_labels(),
                )
                torch.save(self.features, cached_features_file)


    def __len__(self):
        return len(self.features)

    def __getitem__(self, i):
        return self.features[i]

    def get_labels(self):
        return self.corpus.get_labels()

- 원본 문장 : ―효진 역의 김환희(14)가 특히 인상적이었다.
- 레이블한 문장 : ―<효진:PER> 역의 <김환희:PER>(<14:NOH>)가 특히 인상적이었다.

  - PER(인명)으로 레이블링된 효진은 tokens 기준 세번째 토큰(효)부터 네번째 토큰(##진)인 걸 확인할 수 있습니다. 이에 labels에는 세번째 토큰과 네번째 토큰이 PER(인명)이 되도록 합니다. 단 여기에서 B-는 해당 태그의 시작(Begin), I-는 해당 태그의 시작이 아님(Inside)이라는 뜻을 가집니다.

  - PER(인명)으로 레이블링된 김환희는 일곱번째 토큰(김)부터 아홉번째 토큰(##희)인 걸 알 수 있습니다. 이에 labels에는 일곱번째 토큰과 아홉번째 토큰이 PER(인명)이 되도록 합니다. 마찬가지로 14의 경우 labels의 열한번째 토큰이 NOH(기타 수량표현)이 되도록 만들었습니다. 한편 labels에서 O는 outside의 약자로 개체명이 아닌 부분을 의미합니다.

  - 이후 NERDataset은 여기에 인덱싱 작업을 수행하여 input_ids, attention_mask, token_type_ids, labels를 만듭니다. input_ids는 tokens에 인덱싱을 수행한 결과이며 attention_mask는 tokens 각각의 해당 토큰이 패딩인지(0) 아닌지(1)를 나타냅니다. token_type_ids는 세그먼트(segment) 정보로 기본값은 모두 0으로 넣습니다.


- label_ids은 labels의 각 개체명 태그(B-PER, I-PER 등)를 정수로 바꾼 결과

- 개체명 인식을 위한 BERT 모델의 입력은 input_ids, attention_mask, token_type_ids이 되며, 출력은 labels가 되도록 합니다. 

In [13]:
corpus = NERCorpus(args)
train_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

In [17]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=1,
)

In [28]:
value = next(iter(train_dataloader))
print(value)
print(value['labels'].shape)
print(value['input_ids'].shape)
print(value['attention_mask'].shape)
print(value['token_type_ids'].shape)

{'labels': tensor([[ 0,  9, 19,  ...,  2,  2,  2],
        [ 0,  4,  4,  ...,  4,  4,  1],
        [ 0,  8, 18,  ...,  4,  4,  1],
        ...,
        [ 0,  4,  4,  ...,  2,  2,  2],
        [ 0,  4,  4,  ...,  2,  2,  2],
        [ 0,  4,  4,  ...,  4,  4,  1]]), 'input_ids': tensor([[    2, 28413, 22903,  ...,     0,     0,     0],
        [    2, 18368,  8791,  ...,  4474,  8292,     3],
        [    2,  9530,  8242,  ...,  1410,  4700,     3],
        ...,
        [    2,  3108,  4307,  ...,     0,     0,     0],
        [    2,     6, 10747,  ...,     0,     0,     0],
        [    2,   198, 28005,  ...,  4272,  1492,     3]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
 

- RandomSampler 
  - 데이터셋에서 무작위로 데이터를추출하여 미니배치 생성

  - 데이터를 잘 섞어주는 역할을 하며, 학습 시간을 줄이고 높은 정확도를 얻을 수 있음

- SequentialSampler
  - 데이터셋의 인덱스를 처음부터 끝까지 순서대로 반환

  - 데이터셋을 읽는 순서가 중요할 때 사용 (ex, 시계열 데이터)

  

In [31]:
from torch.utils.data import SequentialSampler
val_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=1,
)

---
# Model
---

In [33]:
from transformers import BertConfig, BertForTokenClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForTokenClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the

---
# Train
---